# Hybrid Chatbot: LLM + Code Execution

Este notebook implementa un chatbot híbrido que combina:
1. **LLM** para interpretar preguntas y generar respuestas naturales
2. **Code Execution** para cálculos precisos con pandas
3. **NL Query Classification** para identificar intenciones

**Ventajas:**
- Cálculos 100% precisos
- Respuestas conversacionales naturales
- Escalable a nuevas preguntas

## 1. Configuración del Entorno

In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
from dotenv import load_dotenv
import re
from typing import Dict, List, Any
import json

# Cargar variables de entorno
load_dotenv()

# Verificar API key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("❌ Error: OPENAI_API_KEY no encontrada en el archivo .env")
    client = None
else:
    try:
        client = OpenAI(api_key=api_key)
        print("✅ Cliente OpenAI inicializado correctamente")
    except Exception as e:
        print(f"❌ Error al inicializar OpenAI: {e}")
        client = None

✅ Cliente OpenAI inicializado correctamente


## 2. Clase Hybrid Chatbot

In [2]:
class HybridABTestingChatbot:
    def __init__(self):
        self.data = None
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY')) if os.getenv('OPENAI_API_KEY') else None
        
        # Patrones de preguntas comunes
        self.query_patterns = {
            'total_users': [r'total.*usuario.*control', r'cuántos usuario.*control', r'usuario.*control.*total'],
            'total_users_experiment': [r'total.*usuario.*(experimento_[abc])', r'cuántos usuario.*(experimento_[abc])'],
            'best_experiment': [r'mejor experimento', r'experimento.*mejor', r'más exitoso'],
            'conversion_comparison': [r'conversión.*comparar', r'tasa.*conversión', r'conversion.*rate'],
            'revenue_comparison': [r'revenue.*comparar', r'ingresos.*comparar', r'facturación'],
            'regional_analysis': [r'región', r'regional', r'norte|sur|este|oeste'],
            'store_type_analysis': [r'tipo.*tienda', r'mall|street|outlet'],
            'experiment_count': [r'cuántos.*experimento', r'número.*experimento'],
            'store_count': [r'cuántas.*tienda', r'número.*tienda']
        }
    
    def load_data(self, file_path: str):
        """Carga los datos detallados"""
        if file_path.endswith('.csv'):
            self.data = pd.read_csv(file_path)
        elif file_path.endswith(('.xlsx', '.xls')):
            self.data = pd.read_excel(file_path)
        
        print(f"✅ Datos cargados: {len(self.data)} filas")
        print(f"   Experimentos: {', '.join(self.data['experimento'].unique())}")
    
    def classify_question(self, question: str) -> str:
        """Clasifica la pregunta según patrones predefinidos"""
        question_lower = question.lower()
        
        for intent, patterns in self.query_patterns.items():
            for pattern in patterns:
                if re.search(pattern, question_lower):
                    return intent
        
        return 'general'
    
    def convert_numpy_types(self, obj):
        """Convierte tipos numpy a tipos nativos de Python para JSON"""
        if isinstance(obj, dict):
            return {key: self.convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self.convert_numpy_types(item) for item in obj]
        elif hasattr(obj, 'item'):  # numpy types
            return obj.item()
        elif hasattr(obj, 'tolist'):  # numpy arrays
            return obj.tolist()
        else:
            return obj
    
    # ============ FUNCIONES DE CÁLCULO PRECISAS ============
    
    def calculate_total_users(self, experiment: str = None) -> Dict[str, Any]:
        """Calcula total de usuarios por experimento"""
        if experiment:
            total = int(self.data[self.data['experimento'] == experiment]['usuarios'].sum())
            count = len(self.data[self.data['experimento'] == experiment])
            return {'experiment': experiment, 'total_users': total, 'store_count': count}
        else:
            result = {}
            for exp in self.data['experimento'].unique():
                total = int(self.data[self.data['experimento'] == exp]['usuarios'].sum())
                result[exp] = total
            return result
    
    def calculate_best_experiment(self) -> Dict[str, Any]:
        """Identifica el mejor experimento por métricas clave"""
        summary = self.data.groupby('experimento').agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        
        summary['conversion_rate'] = (summary['conversiones'] / summary['usuarios'] * 100).round(2)
        summary['revenue_per_user'] = (summary['revenue'] / summary['usuarios']).round(2)
        
        # Mejor por revenue per user
        best_experiment = summary['revenue_per_user'].idxmax()
        
        # Convertir todos los datos a tipos nativos
        metrics_dict = {}
        for col in summary.columns:
            metrics_dict[col] = float(summary.loc[best_experiment, col])
        
        all_metrics = {}
        for exp in summary.index:
            exp_metrics = {}
            for col in summary.columns:
                exp_metrics[col] = float(summary.loc[exp, col])
            all_metrics[exp] = exp_metrics
        
        return {
            'best_experiment': best_experiment,
            'metrics': metrics_dict,
            'all_metrics': all_metrics
        }
    
    def calculate_regional_analysis(self, experiment: str = None) -> Dict[str, Any]:
        """Análisis por región"""
        if experiment:
            data_subset = self.data[self.data['experimento'] == experiment]
        else:
            data_subset = self.data
        
        regional_summary = data_subset.groupby(['region']).agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        
        regional_summary['conversion_rate'] = (regional_summary['conversiones'] / regional_summary['usuarios'] * 100).round(2)
        
        # Convertir a tipos nativos
        result = {}
        for region in regional_summary.index:
            region_metrics = {}
            for col in regional_summary.columns:
                region_metrics[col] = float(regional_summary.loc[region, col])
            result[region] = region_metrics
        
        return result
    
    def calculate_store_type_analysis(self, experiment: str = None) -> Dict[str, Any]:
        """Análisis por tipo de tienda"""
        if experiment:
            data_subset = self.data[self.data['experimento'] == experiment]
        else:
            data_subset = self.data
        
        store_summary = data_subset.groupby(['tipo_tienda']).agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        
        store_summary['conversion_rate'] = (store_summary['conversiones'] / store_summary['usuarios'] * 100).round(2)
        
        # Convertir a tipos nativos
        result = {}
        for store_type in store_summary.index:
            store_metrics = {}
            for col in store_summary.columns:
                store_metrics[col] = float(store_summary.loc[store_type, col])
            result[store_type] = store_metrics
        
        return result
    
    def get_experiment_summary(self) -> Dict[str, Any]:
        """Resumen completo de todos los experimentos"""
        summary = self.data.groupby('experimento').agg({
            'usuarios': 'sum',
            'conversiones': 'sum',
            'revenue': 'sum'
        }).round(2)
        
        summary['conversion_rate'] = (summary['conversiones'] / summary['usuarios'] * 100).round(2)
        summary['revenue_per_user'] = (summary['revenue'] / summary['usuarios']).round(2)
        summary['store_count'] = self.data.groupby('experimento').size()
        
        # Convertir a tipos nativos
        result = {}
        for exp in summary.index:
            exp_metrics = {}
            for col in summary.columns:
                value = summary.loc[exp, col]
                exp_metrics[col] = int(value) if col == 'store_count' else float(value)
            result[exp] = exp_metrics
        
        return result
    
    def execute_calculation(self, intent: str, question: str) -> Dict[str, Any]:
        """Ejecuta el cálculo apropiado según la intención"""
        try:
            if intent == 'total_users':
                return self.calculate_total_users('Control')
            
            elif intent == 'total_users_experiment':
                # Extraer experimento específico de la pregunta
                for exp in ['Experimento_A', 'Experimento_B', 'Experimento_C']:
                    if exp.lower() in question.lower():
                        return self.calculate_total_users(exp)
                return self.calculate_total_users()
            
            elif intent == 'best_experiment':
                return self.calculate_best_experiment()
            
            elif intent == 'regional_analysis':
                return self.calculate_regional_analysis()
            
            elif intent == 'store_type_analysis':
                return self.calculate_store_type_analysis()
            
            elif intent in ['conversion_comparison', 'revenue_comparison']:
                return self.get_experiment_summary()
            
            elif intent == 'experiment_count':
                return {'experiment_count': len(self.data['experimento'].unique()),
                       'experiments': list(self.data['experimento'].unique())}
            
            elif intent == 'store_count':
                counts = {}
                for exp in self.data['experimento'].unique():
                    counts[exp] = int(self.data[self.data['experimento'] == exp].shape[0])
                return {'store_counts': counts, 'total_stores': len(self.data)}
            
            else:
                return self.get_experiment_summary()
                
        except Exception as e:
            return {'error': str(e)}
    
    def generate_response(self, question: str, calculation_result: Dict[str, Any], intent: str) -> str:
        """Genera respuesta natural usando LLM"""
        if self.client is None:
            return f"Cálculo completado: {calculation_result}"
        
        system_prompt = """
        Eres un analista experto en A/B testing. Tu trabajo es generar respuestas naturales y profesionales basadas en cálculos ya realizados.
        
        IMPORTANTE:
        - Los cálculos ya están hechos y son 100% precisos
        - NO recalcules ni verifiques números
        - Usa exactamente los números proporcionados
        - Genera una respuesta conversacional y clara
        - Incluye insights y recomendaciones cuando sea apropiado
        """
        
        # Convertir resultado a tipos JSON-serializables
        safe_result = self.convert_numpy_types(calculation_result)
        
        user_prompt = f"""
        PREGUNTA ORIGINAL: {question}
        TIPO DE ANÁLISIS: {intent}
        RESULTADOS CALCULADOS: {json.dumps(safe_result, indent=2, ensure_ascii=False)}
        
        Genera una respuesta natural y profesional basada en estos resultados.
        """
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=600,
                temperature=0.3
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"✅ Cálculo completado: {safe_result}\n\n❌ Error en generación de respuesta: {e}"
    
    def chat(self, question: str) -> str:
        """Función principal del chatbot híbrido"""
        if self.data is None:
            return "❌ Primero carga los datos usando load_data()"
        
        # 1. Clasificar la pregunta
        intent = self.classify_question(question)
        print(f"🎯 Intención detectada: {intent}")
        
        # 2. Ejecutar cálculo preciso
        calculation_result = self.execute_calculation(intent, question)
        print(f"🔢 Cálculo completado: {type(calculation_result).__name__}")
        
        # 3. Generar respuesta natural
        response = self.generate_response(question, calculation_result, intent)
        
        return response
    
    def show_available_patterns(self):
        """Muestra patrones de preguntas soportados"""
        print("🤖 Patrones de preguntas soportados:")
        for intent, patterns in self.query_patterns.items():
            print(f"\n📋 {intent}:")
            for pattern in patterns:
                print(f"   - {pattern}")

## 3. Crear Datos de Prueba

In [ ]:
# Usar los mismos datos que el notebook original
np.random.seed(42)

# Datos detallados por tienda
tiendas_detalle = []
experimentos = ['Control', 'Experimento_A', 'Experimento_B', 'Experimento_C']
base_conversion_rates = [8.5, 9.4, 10.2, 11.9]

for exp, base_conversion in zip(experimentos, base_conversion_rates):
    for i in range(50):  # 50 tiendas por experimento
        tiendas_detalle.append({
            'experimento': exp,
            'tienda_id': f'T_{exp}_{i+1:03d}',
            'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste']),
            'tipo_tienda': np.random.choice(['Mall', 'Street', 'Outlet']),
            'usuarios': np.random.randint(150, 300),
            'conversiones': None,
            'revenue': None,
            'conversion_rate': np.random.normal(base_conversion, 2.5)
        })

tiendas_df = pd.DataFrame(tiendas_detalle)
tiendas_df['conversiones'] = (tiendas_df['usuarios'] * tiendas_df['conversion_rate'] / 100).round().astype(int)
tiendas_df['revenue'] = tiendas_df['conversiones'] * np.random.uniform(15, 25, len(tiendas_df))
tiendas_df['conversion_rate'] = tiendas_df['conversion_rate'].round(2)

# Guardar archivo
tiendas_df.to_csv('hybrid_tiendas_detalle.csv', index=False)

print("✅ Archivo hybrid_tiendas_detalle.csv creado")
print(f"📊 {len(tiendas_df)} filas, {len(experimentos)} experimentos")

## 4. Inicializar Chatbot Híbrido

In [3]:
# Crear chatbot híbrido
hybrid_bot = HybridABTestingChatbot()

# Cargar datos
hybrid_bot.load_data('hybrid_tiendas_detalle.csv')

# Mostrar patrones soportados
hybrid_bot.show_available_patterns()

✅ Datos cargados: 200 filas
   Experimentos: Control, Experimento_A, Experimento_B, Experimento_C
🤖 Patrones de preguntas soportados:

📋 total_users:
   - total.*usuario.*control
   - cuántos usuario.*control
   - usuario.*control.*total

📋 total_users_experiment:
   - total.*usuario.*(experimento_[abc])
   - cuántos usuario.*(experimento_[abc])

📋 best_experiment:
   - mejor experimento
   - experimento.*mejor
   - más exitoso

📋 conversion_comparison:
   - conversión.*comparar
   - tasa.*conversión
   - conversion.*rate

📋 revenue_comparison:
   - revenue.*comparar
   - ingresos.*comparar
   - facturación

📋 regional_analysis:
   - región
   - regional
   - norte|sur|este|oeste

📋 store_type_analysis:
   - tipo.*tienda
   - mall|street|outlet

📋 experiment_count:
   - cuántos.*experimento
   - número.*experimento

📋 store_count:
   - cuántas.*tienda
   - número.*tienda


## 5. Ejemplos de Uso

In [ ]:
# Ejemplo 1: Total usuarios Control
pregunta_1 = "¿Cuántos usuarios en total tienen las tiendas control?"
print(f"🤔 Pregunta: {pregunta_1}")
print(f"💬 Respuesta: {hybrid_bot.chat(pregunta_1)}")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 2: Mejor experimento
pregunta_2 = "¿Cuál es el mejor experimento?"
print(f"🤔 Pregunta: {pregunta_2}")
print(f"💬 Respuesta: {hybrid_bot.chat(pregunta_2)}")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 3: Análisis regional
pregunta_3 = "¿Hay diferencias por región?"
print(f"🤔 Pregunta: {pregunta_3}")
print(f"💬 Respuesta: {hybrid_bot.chat(pregunta_3)}")
print("\n" + "="*80 + "\n")

In [ ]:
# Ejemplo 4: Comparación de conversiones
pregunta_4 = "Compara las tasas de conversión entre experimentos"
print(f"🤔 Pregunta: {pregunta_4}")
print(f"💬 Respuesta: {hybrid_bot.chat(pregunta_4)}")
print("\n" + "="*80 + "\n")

## 6. Chat Interactivo

In [4]:
def chat_interactivo_hybrid():
    """Chat interactivo con el chatbot híbrido"""
    print("🤖 Hybrid Chatbot iniciado!")
    print("💡 Escribe 'salir' para terminar")
    print("📋 Escribe 'patrones' para ver preguntas soportadas\n")
    
    while True:
        pregunta = input("🤔 Tu pregunta: ")
        
        if pregunta.lower() in ['salir', 'exit', 'quit']:
            print("👋 ¡Hasta luego!")
            break
        
        if pregunta.lower() == 'patrones':
            hybrid_bot.show_available_patterns()
            continue
            
        if pregunta.strip() == "":
            continue
            
        print(f"\n💬 Respuesta: {hybrid_bot.chat(pregunta)}")
        print("\n" + "-"*50 + "\n")

# Descomentar para usar el chat interactivo
chat_interactivo_hybrid()

🤖 Hybrid Chatbot iniciado!
💡 Escribe 'salir' para terminar
📋 Escribe 'patrones' para ver preguntas soportadas

🎯 Intención detectada: store_count
🔢 Cálculo completado: dict

💬 Respuesta: En las tiendas de control, hay un total de 50 usuarios en cada una de las cuatro tiendas, lo que suma un total de 200 usuarios en total en las tiendas de control. Esto nos brinda una base sólida para comparar con los resultados de los experimentos A, B y C. Es importante tener en cuenta esta cifra al analizar el impacto de las diferentes estrategias implementadas en los experimentos y para evaluar la efectividad de cada uno en comparación con el grupo de control.

--------------------------------------------------

🎯 Intención detectada: total_users
🔢 Cálculo completado: dict

💬 Respuesta: En las tiendas de control, contamos con un total de 11,196 usuarios distribuidos en 50 tiendas. Esto nos da un promedio de aproximadamente 224 usuarios por tienda. Es importante seguir monitoreando el comportamiento 

## 7. Verificación de Precisión

In [ ]:
# Verificar cálculos manualmente
print("🔍 Verificación de precisión:")

# Total usuarios Control - cálculo manual
manual_control_users = tiendas_df[tiendas_df['experimento'] == 'Control']['usuarios'].sum()
print(f"📊 Usuarios Control (manual): {manual_control_users}")

# Total usuarios Control - chatbot híbrido
hybrid_result = hybrid_bot.calculate_total_users('Control')
print(f"🤖 Usuarios Control (híbrido): {hybrid_result['total_users']}")
print(f"✅ Coinciden: {manual_control_users == hybrid_result['total_users']}")

# Resumen por experimentos
print("\n📈 Resumen por experimento:")
summary = hybrid_bot.get_experiment_summary()
for exp, metrics in summary.items():
    print(f"   {exp}: {metrics['usuarios']} usuarios, {metrics['conversiones']} conversiones, {metrics['conversion_rate']}% conv. rate")